<a href="https://colab.research.google.com/github/luisdi97/Proyecto_PF-3347/blob/main/proyecto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Proyecto Final Análisis en Ciencia de Datos

### Estudiantes

* Luis Diego Araya Campos
* Cristian Alejandro Herrera Barboza
* Josué Raúl Rivas Muñoz

# 1. Selección del Dataset
Este dataset ha sido seleccionado debido a su relevancia en el contexto de la planificación energética y su aplicabilidad en la creciente adopción de energías renovables, especialmente la energía solar. Dado que uno de los integrantes del grupo tiene acceso a la información requerida, y tomando en cuenta que el resultado ayudará a la toma de decisiones del proceso actual se analiza como una excelente opción que permitirá optimizar procesos y ayudará en la toma de decisiones que podrían fomentar la eficiencia energética y la sostenibilidad en comunidades locales.

# 2. Descripción del Problema

### Contexto del problema
En el contexto de la transición hacia energías renovables, como la solar, el análisis de la capacidad de instalación de paneles solares en distintos puntos de una red eléctrica es crucial para optimizar el consumo y la generación de energía sostenible. Sin embargo, esta capacidad no es uniforme y depende de varios factores de la red, tales como la capacidad del transformador, la carga actual en kVA, y la distancia de los puntos de carga respecto a la subestación y al transformador.

Para determinar la viabilidad de la instalación de paneles solares, existe un valor que define el límite máximo de generación de energía que puede ser inyectada en la red sin comprometer la estabilidad del sistema ni exceder la capacidad de los componentes eléctricos, como transformadores y líneas de transmisión, el cual por medio de algoritmos de machine learning se pretende identificar cual la máxima capacidad de corriente en kW que soporta una ubicación donde se pretende instalar un panel.

#### Importancia:
Resolver este problema tiene un impacto directo en la capacidad de planificación y expansión de infraestructuras de energías renovables. La solución que genera el poder analizar y procesar estos datos por medio de modelos de aprendizaje permite mejorar la planificación de la red y así de forma inteligente, da la posibilidad de mostrar como puede expandirse el uso de paneles de red en diferentes nodos.
Por otro lado al permitir la integración de paneles solares de forma segura y óptima reduce la huella de carbono y promueve un uso más ecológico de los recursos eléctricos esto a mismo tiempo que se optimizan los sistemas actuales.

#### Objetivo del Análisis

El objetivo de este análisis es desarrollar un modelo predictivo que estime el "IC Max per Load" en función de variables clave de la red. Este modelo permitirá determinar la capacidad de instalación de paneles solares en cada nodo de carga, facilitando así la toma de decisiones informadas sobre la incorporación de fuentes de energía renovable en cada punto de la red. Al optimizar la asignación de estas instalaciones, se espera que se logren beneficios económicos y ambientales significativos, ayudando a reducir la dependencia de fuentes de energía no renovables y mejorando la eficiencia de la red eléctrica.

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import root_mean_squared_error

In [56]:
df = pd.read_csv('datos_IA_ICA.csv')
df

,Unnamed: 0,Node Id,Network Id,IC Max per Load (kW),Constraint per Load,IC Volt Variation MT (kW),IC Volt Variation BT (kW),IC Voltage Unbalance (kW),IC Volt Limits (kW),IC Loading (kW),...,Distributed Generation per Load (kW),IC Max per Transfomer (kW),Constraint per Transformer,Transformer Load (kVA),Number of Load Nodes,Distributed Generation per Transformer (kW),Distance from substation to Load (m),Distance from transformer to Load (m),Max distance from transformer to Load (m),IC Prot Reach (kW)
0,0,FTSC_460401_22050379217-1111301_4109611586_LOAD,206-1,66.227012,IC Loading (kW),92.834901,100.000000,100.0,100.000000,66.227012,...,0.00,66.227012,IC Loading (kW),46.701847,1,0.00,6231.403964,0.0,0.0,NaN
1,1,FTSC_460412_67034647707-1111234_7081091106_LOAD,206-1,39.338502,IC Loading (kW),44.136866,76.979637,100.0,55.816274,39.338502,...,0.00,39.338502,IC Loading (kW),15.867834,1,0.00,6299.541223,0.0,0.0,NaN
2,2,FTSC_460412_85442652274-1111233_5866960716_LOAD,206-1,32.842440,IC Loading (kW),42.906003,74.829132,100.0,76.850072,32.842440,...,0.00,32.842440,IC Loading (kW),23.350922,1,0.00,6300.676739,0.0,0.0,NaN
3,3,FTSC_460423_2939438168-1111168_2222679332_LOAD,206-1,32.842441,IC Loading (kW),42.897159,74.813462,100.0,76.875324,32.842441,...,0.00,32.842441,IC Loading (kW),23.350922,1,0.00,6366.654858,0.0,0.0,NaN
4,4,FTSC_460579_63665874675-1108490_3345925752_LOAD,206-1,13.608194,IC Loading (kW),18.157137,31.730709,100.0,19.124799,13.608194,...,0.00,13.608194,IC Loading (kW),4.294494,1,0.00,9189.457160,0.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83561,83561,BT_SUB_523315_16480317805-1129377_9141547661_LOAD,707-4,95.966316,IC Loading (kW),1000.000000,422.193711,NaN,539.364952,95.966316,...,0.00,95.966316,IC Loading (kW),45.856394,2,0.00,11443.200000,8.8,8.8,NaN
83562,83562,BT_SUB_523321_5470127044-1129376_1804415341_LOAD,707-4,133.749377,IC Loading (kW),1000.000000,450.027015,NaN,609.635581,133.749377,...,0.00,95.966316,IC Loading (kW),45.856394,2,0.00,11442.100000,7.7,8.8,NaN
83563,83563,BT_SUB_523575_9945391603-1127846_209566636_LOAD,707-4,247.742728,IC Loading (kW),1000.000000,1000.000000,NaN,490.543335,247.742728,...,0.00,247.742728,IC Loading (kW),103.115261,1,0.00,17169.400000,2.8,2.8,NaN
83564,83564,BT_SUB_523580_75080993865-1128024_0064656883_LOAD,707-4,14.863747,IC Loading (kW),1000.000000,553.675056,NaN,579.989884,14.863747,...,105.05,14.863747,IC Loading (kW),29.009023,1,105.05,17001.700000,2.5,2.5,NaN


In [57]:
df_final = df.copy()

df_final['Num_Nodos_Load'] = df_final.groupby('Network Id')['Network Id'].transform('count')

In [58]:
df_final = df_final.loc[df_final['Distributed Generation per Load (kW)'] == 0.0].copy()

In [59]:
df_final = df_final.loc[df_final['Load (kVA)'] >= 0.0].copy()

In [60]:
kVA_transformers = list(df_final['Transformer Capacity (kVA)'].unique())
kVA_transformers.sort()
kVA_transformers

[10.0,
 15.0,
 20.0,
 25.0,
 30.0,
 32.5,
 35.0,
 37.5,
 40.0,
 45.0,
 50.0,
 52.5,
 55.0,
 60.0,
 62.5,
 65.0,
 75.0,
 85.0,
 87.5,
 100.0,
 110.0,
 112.5,
 113.0,
 115.0,
 125.0,
 137.5,
 150.0,
 160.0,
 167.0,
 175.0,
 185.0,
 200.0,
 225.0,
 250.0,
 300.0,
 310.0,
 400.0,
 500.0,
 501.0,
 511.0,
 750.0,
 999.0,
 1000.0,
 1500.0,
 2000.0,
 2500.0,
 3000.0]

In [61]:
df_final = df_final.loc[df_final['Transformer Capacity (kVA)'] <= 50.0].copy()

In [62]:
features = [
    'Transformer Capacity (kVA)',
    'Total Path Z (Ω)',
    'Load (kVA)',
    'Number of Load Nodes',
    'Distance from substation to Load (m)',
    'Distance from transformer to Load (m)',
    'Num_Nodos_Load',
]

target = 'IC Max per Load (kW)'

In [63]:
X = df_final.loc[:, features].copy()
y = df_final.loc[:, target].copy()

In [64]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

params = {
    # "loss": "squared_error",
    "learning_rate": 0.01,
    "n_estimators": 500,
    # "subsample": 1.0,
    # "criterion": "friedman_mse",
    "min_samples_split": 5,
    # "min_samples_leaf": 1,
    # "min_weight_fraction_leaf": 0.0,
    "max_depth": 4,
    # "min_impurity_decrease": 0.0,
    # "init": None,
    # "random_state": None,
    # "max_features": None,
    # "alpha": 0.9,
    # "verbose": 0,
    # "max_leaf_nodes": None,
    # "warm_start": False,
    # "validation_fraction": 0.1,
    # "n_iter_no_change": None,
    # "tol": 0.0001,
    # "ccp_alpha": 0.0,
}

In [65]:
reg = GradientBoostingRegressor(**params)
reg.fit(X_train, y_train)

GradientBoostingRegressor(learning_rate=0.01, max_depth=4, min_samples_split=5,
                          n_estimators=500)

In [66]:
rmse = root_mean_squared_error(y_test, reg.predict(X_test))
print("The root mean squared error (RMSE) on test set: {:.4f}".format(rmse))

The root mean squared error (RMSE) on test set: 3.9780
